# テーマ  
### ・最小全域木  
### ・トポジカルソート

## 全域木  
グラフにおいて、すべての頂点がつながっている木(閉路を持たない)  
最小全域木：全域木の中での篇の距離の総和が最小になるもの  
・辺ベースのアプローチ（クラスカル法）
・ノードベースのアプローチ（プリム法）

### クラスカル法  
存在する辺を距離の短い順に並べて順に⼊れていき，閉路が出来ないことが確認できた場合は追加し，全部の辺をチェックしたら終了    
#1 すべての辺を距離の短い順にソート  
#2 一番距離の短い辺からスタート  
#3 今までにできた木に辺を追加したとき、閉路が新しくできないことを確認する  
#4 すべての辺をチェック  
→#3に時間を要してしまう  
##### 素集合データ構造(Union-Find木)  
要素を素集合（互いに重ならない集合）に分割して管理するデータ構造．このデータ構造には2つの操作がある．  
Unite：2つの集合をマージする  
Find：ある要素がどの集合にいるかを⾒つける．  
<実装>
N個の要素がある時，⻑さNの配列を⽤意  
この配列には親ノードのindexを⼊れる．⾃分が根ノードの場合は⾃分⾃⾝のindexを⼊れる  
この値をたどっていけば最終的に根ノードに⾏き着く  
最初の時点では⾃分⾃⾝しかグループに属していないので，⾃分⾃⾝が根ノードになる  

#1 Unite時に気の高さが高いほうにマージ  
#2 根を調べたときに直接根につながるようにつなぎ変える


In [1]:
#実装例
class UnionFind:
    def __init__(self,n):
        self.parent=[i for i in range(n)]
        self.height=[0 for _ in range(n)]
    
    def get_root(self,i):
        if self.parent[i]==i:
            return i
        else:
            self.parent[i]=self.get_root(self.parent[i])
            return self.parent[i]
    
    def unite(self,i,j):
        root_i=self.get_root(i)
        root_j=self.get_root(j)
        if root_i!=root_j:
            if self.height[root_i]<self.height[root_j]:
                self.parent[root_i]=root_j
            else:
                self.parent[root_j]=root_i    
                if self.height[root_i]==self.height[root_j]:
                    self.height[root_i]+=1
                    
    def is_in_group(self,i,j):
        if self.get_root(i)==self.get_root(j):
            return True
        else:
            return False


計算量はアッカーマン関数の逆関数になることが知られており定数倍とみなして扱われることもある。  
Union-Find木では閉路の判定は二つのノードが同じグループに属していないことをチェックすればよく速攻判定可能!  


In [71]:
#クラスカル法の実装例
def kruskal(V,e_list):
    e_cost_sorted=[]
    #ソートのために先頭を距離にする
    for e in e_list:
        e_cost_sorted.append([e[2],e[1],e[0]])
    e_cost_sorted.sort()
    #Union-Find
    uf_tree=UnionFind(V)
    #最小全域木の辺を保持するリスト
    mst=[]
    #距離の小さい順にみていきe[1],e[2]がおなじグループでないのなら同じグループにする
    for edge in e_cost_sorted:
        cost,u,v=edge
        if uf_tree.is_in_group(u,v)==False:
            uf_tree.unite(u,v)
            mst.append([u,v])
    mst.sort()
    print(mst)
    

In [72]:
edges_list = [[0, 1, 5], [0, 2, 4], [1, 0, 5], [1, 3, 3], [1, 5, 9], [2, 0, 4], [2, 3, 2], [2, 4, 3], [3, 1, 3], [3, 2, 2], [3, 6, 7], [3, 7, 5], [4, 2, 3], [4, 6, 8], [5, 1, 9], [6, 3, 7], [6, 4, 8], [6, 7, 1], [7, 3, 5], [7, 6, 1]] 
kruskal(8, edges_list)

[[0, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [6, 7]]


##### クラスカル法の計算量  
隣接リストの場合、辺の数を$|E|$として辺のソートに$O(|E|log|E|)$かかる  
各辺を入れるかどうかの判断はUnion-Find木を用いれば$O(\alpha(|V|))$となりこれを$O(|E|)$回行う。  
よってアルゴリズム全体の計算量としては$O(|E|log|E|)$

### プリム法  
すでに到達した頂点の集合からまだ到達していない頂点の集合への辺のうち，距離が最短のものを追加し，全ノードつながったら終了  
#1 最初のノードを1つ選び（どれでも可），訪問済にする  
#2 そのノードに繋がっている全ての辺を取り，最⼩全域⽊の候補の辺に⼊れる
#3  最⼩全域⽊の候補の辺の中から，接続先のノードが未訪問である最短の距離の辺を選ぶ．（接続先のノードが訪問済の場合は無視して次候補に移る．）  
#4 選んだ辺を最⼩全域⽊に⼊れ，その接続先にあるノードを訪問済にする   
#5  #4で新しく訪問したノードから，更にその先につながっている辺のうち，接続先のノードが未訪問の全ての辺を最⼩全域⽊の候補に⼊れる  
#6 以降，全ノードが訪問済になるまで#2〜#4を繰り返す  
<計算量>  
ノードの数を$|V|$として  
隣接行列+最短の辺の単純な探索:$O(|V|^3)$(最短の辺の探索に$O(|V|^2)$)  
隣接リスト+最短の辺の単純な探索:$O(|E||V|)$  
ただしデータ構造を工夫することで高速化できる  
以下の実装例は隣接リスト+ヒープを使う

In [10]:
import heapq
inf=10**10
def prim(V,e_list):
    edges_from=[[] for _ in range(V)]
    for e in e_list:
        edges_from[e[0]].append([e[2],e[0],e[1]])
    e_heapq=[]
    mst=[]
    #ノードが最小全域木に入ったかどうかのフラグ
    included=[False]*V
    #ノードをまず一つ選ぶ
    included[0]=True
    #ノード0に接続する辺をヒープに入れる
    for e in edges_from[0]:
        heapq.heappush(e_heapq,e)
                
    while len(e_heapq):
        min_edge=heapq.heappop(e_heapq)
        #その辺の到達先が未訪問なら追加
        if not included[min_edge[2]]:
            included[min_edge[2]]=True
            mst.append([min_edge[1],min_edge[2]])
            for e in edges_from[min_edge[2]]:
                heapq.heappush(e_heapq,e)
    mst.sort()
    print(mst)

In [11]:
edges_list = [[0, 1, 5], [0, 2, 4], [1, 0, 5], [1, 3, 3], [1, 5, 9], [2, 0, 4], [2, 3, 2], [2, 4, 3], [3, 1, 3], [3, 2, 2], [3, 6, 7], [3, 7, 5], [4, 2, 3], [4, 6, 8], [5, 1, 9], [6, 3, 7], [6, 4, 8], [6, 7, 1], [7, 3, 5], [7, 6, 1]] 
prim(8, edges_list)

[[0, 2], [1, 5], [2, 3], [2, 4], [3, 1], [3, 7], [7, 6]]


##### プリム法の計算量（ヒープ使用時）
ヒープに入る要素の数は辺の総数で追加、削除にかかる計算量は$O(log|E|)$  
追加も削除も$O(|E|)$回あるので全体の計算量は$O(|E|log|E|)$となる。  
フィボナッチヒープを用いることで$O(|E|+|V|log|V|)$に落とせる。

## トポジカルソート  
閉路の無い有向グラフを「ソート」する．全ての有向辺が1つの向きになるようにノードを並び替える．このようなグラフを有向⾮巡回グラフと呼ぶ．英語ではDirected Acyclic Graph（DAG）．また，与えられるDAGには多重辺がないとする．  
##### 有向木  
根である点をただ1つ持ち，辺の向きが根から葉，もしくは葉から根のどちらかに限られる．辺の向きが全て同じでない場合，「任意の2点を結ぶ経路がただ1つ存在する」という⽊の制約を満たさない．有向⽊はDAGである．ただしDAGは必ずしも有向木ではない.  
##### 次数. 入次数  
次数:あるノードにつながっている辺の総数
入次数:あるノードに入ってくる辺の数 ⇔ 出次数:出ていく辺の数  
[次数]=[入次数]+[出次数]　　
### Kahnのトポロジカルソートの方針
⼊次数0のノードを⾒つけ出し，それをグラフから取り除き，ソート済の場所に⼊れていく．  　　
具体的な実装としては，各ノードの⼊次数を予め記録しておき，ノードを取り出すたびに⼊次数の値を更新する．  　　
この時，取り除いたノードの出⼒辺の接続先ノードの⼊次数のみ更新すれば良い．  


In [20]:
#実装例
V=5
E=6
edges = [[0, 1], [0, 2], [1, 3], [2, 3], [2, 4], [3, 4]]

from collections import deque
def KahnTopo(V,E,edges):
    indeg=[0]*V #入次数
    outedge=[[] for _ in range(V)] #出力辺
    for v_from,v_to in edges:
        indeg[v_to]+=1
        outedge[v_from].append(v_to)
    #ソート済のノードを格納する配列
    sorted_g=list(v for v in range(V) if indeg[v]==0)
    #入次数0のノードを処理するためのdeque
    deq=deque(sorted_g)
    while deq:
        v=deq.popleft()
        for u in outedge[v]:
            E-=1
            indeg[u]-=1
            if indeg[u]==0:
                deq.append(u)
                sorted_g.append(u)
    
    if E!=0:
        return -1
    return sorted_g

print(KahnTopo(V,E,edges))

[0, 1, 2, 3, 4]


In [23]:
import heapq
def KahnTopo_1(V,E,edges):
    indeg=[0]*V #入次数
    outedge=[[] for _ in range(V)] #出力辺
    for v_from,v_to in edges:
        indeg[v_to]+=1
        outedge[v_from].append(v_to)
    #ソート済のノードを格納する配列
    sorted_g=list(v for v in range(V) if indeg[v]==0)
    #入次数0のノードを処理するためのdeque
    min_heap=[]
    for v in range(V):
        if indeg[v] == 0:
            heapq.heappush(min_heap, v)
    
    result = []
    while min_heap:
        v = heapq.heappop(min_heap)
        result.append(v)
        for u in outedge[v]:
            E -= 1
            indeg[u] -= 1
            if indeg[u] == 0:
                heapq.heappush(min_heap, u)
    
    if E != 0:
        return -1
    return result

V=4 
E=3
edges=[[0, 1],[1, 3],[2, 3]]
print(KahnTopo_1(V,E,edges))

[0, 1, 2, 3]


### Tarjanのトポジカルソート  
ノードを一つ選び、DFSをたどっていく. 先に進めないところまで到達したら，後戻りしながらソート済の場所に先頭から順に⼊れていく

In [13]:
#実装例
def TarjanTopo(V,edge):
    def check(v):
        if visited[v]==1:
            return ValueError
        elif visited[v]==0:
            visited[v]=1
            for to_v in outedge[v]:
                check(to_v)
            visited[v]=2
            sorted_g.appendleft(v)
    #ノードをすでに見たかどうかを格納する配列
    visited=[0]*V
    outedge=[[] for _ in range(V)]
    for e in edges:
        outedge[e[0]].append(e[1])
    sorted_g=deque()
    for i in range(V):
        check(i)
    return sorted_g

V = 5 
edges = [[0, 1], [0, 2], [1, 3], [2, 3], [2, 4], [3, 4]] 
print(TarjanTopo(V, edges))

deque([0, 2, 1, 3, 4])


#### トポジカルソートの計算量  
ソートの本質的な部分はKahnのアルゴリズムの場合whileループ、Tarjamもアルゴリズムの場合再帰部分になる  
入力されたグラフがDAGの場合、すべてのノードと辺は高々一回しかチェックされないため、$O(|V|+|E|)$になる  
<応用例>  
##### 閉路チェック  
有向グラフにおいてトポロジカルソートできなかった場合，どこかに閉路が存在する（有向巡回グラフになっている）．  
##### 最短経路  
トポロジカルソートを実⾏し，そのソート順にノードを⾒ていくことで，最短経路を求めることができる  
トポロジカルソート結果順にノードを調べていれば，今チェックしているノードに⼊ってくる経路は全て調べ終えた状態になっているはずで，そのノードに⾄る最短経路が既に計算されていることになる．  


In [19]:
def shortest_path(V,E,edges,start_vertex):
    #隣接リスト
    edge_list=[[] for _ in range(V)]
    for u,v,weight in edges:
        edge_list[u].append([v,weight])
    sorted_g = KahnTopo(V, E, [(u, v) for u, v, _ in edges])
    if not sorted_g:
        return ValueError
    #最短距離を保持するリストの初期化
    dist=[float('inf')]*V
    dist[start_vertex]=0
    for u in sorted_g:
        if dist[u]!=float('inf'):
            for v,weight in edge_list[u]:
                if dist[v]>dist[u]+weight:
                    dist[v]=dist[u]+weight
    return dist

V = 5 
E = 6 
edges = [[0, 1, 5], [0, 2, 6], [1, 3, 8], [2, 3, 2], [2, 4, 5], [3, 4, 1]] 
print(shortest_path(V, E, edges, 0))

[0, 5, 6, 8, 9]


トポジカルソートでDAGの最短経路を求める場合の計算  
トポジカル部分は$O(|V|+|E|)$  
最短経路をDPで求める部分では各ノードｈソートされた順に一回チェックされ、確変も一回だけ処理される。$O(|V|+|E|)$  


In [38]:
#Extra
class UnionFind:
    def __init__(self,n):
        self.parent=list(range(n))
        self.height=[0]*n
        self.max_edge=[0]*n
    
    def get_root(self,x):
        if self.parent[x] != x:
            root=self.get_root(self.parent[x])
            self.max_edge[x]=max(self.max_edge[x],self.max_edge[self.parent[x]])
            self.parent[x]=root
        return self.parent[x]
    
    def unite(self,x,y,cost):
        root_x=self.get_root(x)
        root_y=self.get_root(y)
        if root_x!=root_y:
            if root_x==0:
                self.parent[root_y]=root_x
                self.max_edge[root_y]=max(self.max_edge[root_y],cost)
            elif root_y==0:
                self.parent[root_x]=root_y
                self.max_edge[root_x]=max(self.max_edge[root_x],cost)
            elif self.height[root_x]<self.height[root_y]:
                self.parent[root_x]=root_y
                self.max_edge[root_x]=max(self.max_edge[root_x],cost)
            else:
                self.parent[root_y]=root_x
                self.max_edge[root_y]=max(self.max_edge[root_y],cost)
                if self.height[root_x]==self.height[root_y]:
                    self.height[root_x]+=1
    
    def is_in_group(self, x, y):
        return self.get_root(x) == self.get_root(y)
    
    
    def max_edge_in_path(self, x):
        max_edge=0
        while self.parent[x]!=x:
            max_edge=max(max_edge,self.max_edge[x])
            x=self.parent[x]
        return max_edge

def kruskal(n,edges):
    uf_tree=UnionFind(n)
    edges.sort()
    mst_cost=0
    for cost,u,v in edges:
        if not uf_tree.is_in_group(u,v):
            uf_tree.unite(u,v,cost)
            mst_cost+=cost
    return mst_cost,uf_tree

def solve(n,edges,queries):
    edges_with_costs=[(cost, u-1, v-1) for u,v,cost in edges]
    mst_cost,uf_tree=kruskal(n, edges_with_costs)
    results=[]
    for x,c in queries:
        x-=1
        max_edge=uf_tree.max_edge_in_path(x)
        new_cost=mst_cost+c-max_edge
        results.append(new_cost)
    return results


results = solve(N, edges_list, query_list)
for result in results:
    print(result)


46024105078
46468702086
46047847570
46002244369
46684452930
46020481355
46680978402
46784352694
46548015686
46344794919
46081736844
46756248140
46284608464
46758495274
46075808948
45984208675
46398407460
46198371472
46169300210
46383180071
46316392049
46554212975
46682675057
46000526965
46598476473
46516291108
46580185714
46843374648
46576998306
46201858806
46603476007
45943931539
46072814927
46108863013
46693681624
46734476465
46092509271
46880668071
46428157135
46159134318
46098225049
46288992941
46742989835
46119568178
46863673034
46623321680
46257393104
46624222422
46258879615
46253829146
46123957557
46093213846
46520924263
46242910427
46353440046
46797987286
46863509844
46153440748
46707497826
46774234103
46425108668
46056832973
46293422080
46892021983
46371637540
46327514497
47386339098
46422087441
46032109877
45959881075
45972576644
46137971055
46664472906
46319132851
46384905480
46251449466
46389155063
46780764625
45985327291
46291273083
46738399468
46785805260
46374174782
4685